In [1]:
# all codes come are changed based on LDR code.
# 用添加噪声的数据进行训练
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import get_noise_uniform
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# 导入mnist数据集，进行对比
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'mnist'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

In [3]:
# data_preprocessing and parameters
import time
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET2 = 'FMNIST'
ROOT2    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader2, dataset2 = get_dataloader(
        data_name=DATASET2,
        root=ROOT2,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

data_time = time.time()
iter_dataloader2 = iter(dataloader2)
data2, label2 = next(iter_dataloader2)
data_time = time.time() - data_time
real_cpu2 = data2.to(device)
real_label2 = label2.clone().detach()

In [4]:
# direct use real_label,normalize var as 0.5
fake_noise_normal = torch.from_numpy(np.random.normal(loc=0.5, scale=0.5, size=[2048,1,32,32])) # var is 1/4
fake_noise_normal = fake_noise_normal.type(torch.FloatTensor)
fake_noise_normal = fake_noise_normal.to(device)
fake_label = torch.from_numpy(np.random.randint(10,size=[2048]))

fake_noise_uniform = np.sqrt(3)*torch.from_numpy(np.random.uniform(0,1,[2048,1,32,32])) # 1/12
fake_noise_uniform = fake_noise_uniform.type(torch.FloatTensor)
fake_noise_uniform = fake_noise_uniform.to(device)

fake_noise_cauchy = torch.from_numpy(np.random.standard_cauchy(size=[2048,1,32,32])+0.5)
fake_noise_cauchy = fake_noise_cauchy.type(torch.FloatTensor)
fake_noise_cauchy = fake_noise_cauchy.to(device)

fake_noise_laplace = torch.from_numpy(np.random.laplace(loc=0.5, scale=1.0, size=[2048,1,32,32]))/ np.sqrt(8) # var is 2
fake_noise_laplace = fake_noise_laplace.type(torch.FloatTensor)
fake_noise_laplace = fake_noise_laplace.to(device)

fake_noise_gumbel = torch.from_numpy(np.random.gumbel(loc=0.5,scale=1.0,size=[2048,1,32,32]))/ np.sqrt(6.58) #pi^2/6 1.645
fake_noise_gumbel = fake_noise_gumbel.type(torch.FloatTensor)
fake_noise_gumbel = fake_noise_gumbel.to(device)
# from 0.01 - 0.99,0 -1
epsilon_noise = 0.1

In [5]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict

# generate different distribution via gaussian init and uniform
def get_cauchy_noise(netD,net_noise1,net_noise2):
    models =[netD,net_noise1, net_noise2]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[1][key] / worker_state_dict[2][key]
    return update_dict
        

In [6]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
epsilon_noise = 0.1
num_steps = 1000
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []


transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.MNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10



In [7]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius_noise(radius1,radius2,netD,noise, real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2


def gs_radius2(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    net_noise1 = get_noise(DATASET, device, LABEL)
    net_noise_cauchy = get_cauchy_noise(netD,net_noise,net_noise1)
    net_noise.load_state_dict(net_noise_cauchy)
    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius3(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise_uniform(DATASET, device, LABEL)

    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [8]:
#acc at first for NetD, there exist some issues for netG here, resample will get it right.
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.53s/it]


----------------------------
Train_z, Test_z
PCA: 0.9526
SVD: 0.9485
NetD, Acc at first 0.9538


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.32s/it]


----------------------------
Train_z, Test_z
PCA: 0.9354
SVD: 0.9327
NetG, Acc at first 0.9627


In [9]:
# data selected
mean_V = []
n_dis2 = 1
n_dis = 1
mode = 11
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
loss_V = 0
last_V = -1000
num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 1
selected_radius = 0.7
#data_list = [real_cpu,fake_noise_normal,fake_noise_uniform,fake_noise_laplace,fake_noise_cauchy,fake_noise_gumbel, new_sample,new_sample2,new_sample3]
TT = 0
TF = 0
FT = 0
FF = 0
data_list = [real_cpu,real_cpu2,fake_noise_normal]
label_list = [real_label,real_label2,real_label2]
start_epoch = 20
for epoch in range(num_steps):
    if epoch < start_epoch:

        for i in range(n_dis):
            netD.zero_grad()
            optD.zero_grad()
            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
            print(errD, errD_EC)
            errD.backward()
            optD.step()
        temp = 0
        for j in range(n_dis2):
            netG.zero_grad()
            optG.zero_grad()

            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG, errG_EC = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            print(errG, errG_EC)
            temp = errG
            errG = (-1) * errG
            errG.backward()
            optG.step()

            update_dic = update_netV(netV,netG,net_noise,fed_state_dict,radius)
            netV.load_state_dict(update_dic)

        loss_V = temp
        
    else:
        print('epoch',epoch)
        pos = random.choice(range(3))
        radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=data_list[pos],real_label=label_list[pos],tor=1e-5,n_dis=2,lam=0.618,mode=mode)
        print(radius)
        print(pos)
        if radius > selected_radius:
            print('This data is wrong to this model, wrong distribution,pass')
            if pos ==0:
                TF +=1
            else:
                FF +=1
            pass
        else:
            radius_list.append(radius)
            if pos == 0:
                TT += 1
            else:
                FT += 1
            for i in range(n_dis):
                netD.zero_grad()
                optD.zero_grad()
                Z = netD(data_list[pos])
                net_noise = get_noise(DATASET, device, LABEL)
                fed_state_dict = get_netG(netD,netV,net_noise,radius)
                netG.load_state_dict(fed_state_dict)
                Z_bar = netG(data_list[pos])
                errD, errD_EC = mcr_gan_loss(Z, Z_bar, label_list[pos], 1, n_dis)
                print(errD, errD_EC)
                errD.backward()
                optD.step()
            temp = 0
            for j in range(n_dis2):
                netG.zero_grad()
                optG.zero_grad()

                Z = netD(data_list[pos])
                net_noise = get_noise(DATASET, device, LABEL)
                fed_state_dict = get_netG(netD,netV,net_noise,radius)
                netG.load_state_dict(fed_state_dict)
                Z_bar = netG(data_list[pos])
                errG, errG_EC = mcr_gan_loss(Z, Z_bar, label_list[pos], n_dis2 - 1, n_dis2)
                print(errG, errG_EC)
                temp = errG
                errG = (-1) * errG
                errG.backward()
                optG.step()

                update_dic = update_netV(netV,netG,net_noise,fed_state_dict,radius)
                netV.load_state_dict(update_dic)
            


99.98928869049641
tensor(-10.9085, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-3.6057, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.7505, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.8966, device='cuda:0', grad_fn=<AddBackward0>)]
10.846907917541133
tensor(-10.4577, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.3734, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.5399, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.6157, device='cuda:0', grad_fn=<AddBackward0>)]
99.96289030281403
tensor(-10.2805, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.3734, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3791, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.0563, device='cuda:0', grad_fn=<AddBackward0>)]
1.8151067581511018
tensor(-18.7933, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-8.1306, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.9333, device='cuda:0', grad_fn=<NegBackward0>), tensor(-40.6549, device='cuda:0', grad_fn=<AddBac

0.5031080799933816
tensor(-48.1850, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.4603, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.6015, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.3582, device='cuda:0', grad_fn=<AddBackward0>)]
0.5349005154498343
tensor(-48.4229, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-23.4603, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.7491, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.1471, device='cuda:0', grad_fn=<AddBackward0>)]
0.5099231202132696
tensor(-50.2844, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.4513, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.7171, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.2979, device='cuda:0', grad_fn=<AddBackward0>)]
0.5389800370001842
tensor(-50.3100, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.4513, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.6186, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.3943, device='cuda:0', grad_fn=<

2.4328206917047677
2
This data is wrong to this model, wrong distribution,pass
epoch 57
0.45042160603504244
0
tensor(-59.0506, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-28.8580, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.2888, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.7109, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-59.6809, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.3721, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.3842, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.8520, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 58
5.423230924337307
2
This data is wrong to this model, wrong distribution,pass
epoch 59
1.243184067231848
1
This data is wrong to this model, wrong distribution,pass
epoch 60
1.0416662698974406
1
This data is wrong to this model, wrong distribution,pass
epoch 61
1.6607779135291392
2
This data is wrong to this model, wrong distribution,pass
epoch 62
1.0298836735005736
1
This data is wrong to this model, wrong 

epoch 90
1.1633412279511375
1
This data is wrong to this model, wrong distribution,pass
epoch 91
2.2167165336288672
2
This data is wrong to this model, wrong distribution,pass
epoch 92
1.146914712986747
1
This data is wrong to this model, wrong distribution,pass
epoch 93
0.40271037993089503
0
tensor(-65.5035, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.3069, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.4910, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.5053, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-65.3981, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.2904, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.4218, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3974, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 94
1.2304490394074357
1
This data is wrong to this model, wrong distribution,pass
epoch 95
2.721667863907938
2
This data is wrong to this model, wrong distribution,pass
epoch 96
1.1124678358601974
1
This data is wrong to this mode

epoch 127
0.39938658136194405
0
tensor(-70.2017, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.8226, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.6753, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.4948, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-69.9120, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.5828, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.6972, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.1142, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 128
0.4179443903342805
0
tensor(-69.9857, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.5828, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.7047, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.4641, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-70.5684, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.9137, device='cuda:0', grad_fn=<NegBackward0>), tensor(-33.9528, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.4839, device='cuda:0', grad_fn=<AddBackward0>

...

epoch 342
1.1149899837275992
1
This data is wrong to this model, wrong distribution,pass
epoch 343
2.1092735116259065
2
This data is wrong to this model, wrong distribution,pass
epoch 344
0.9979191544829811
1
This data is wrong to this model, wrong distribution,pass
epoch 345
1.0276104015651186
1
This data is wrong to this model, wrong distribution,pass
epoch 346
0.29595135493657443
0
tensor(-84.9557, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-42.2260, device='cuda:0', grad_fn=<NegBackward0>), tensor(-41.6432, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.9641, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-84.7841, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-42.1583, device='cuda:0', grad_fn=<NegBackward0>), tensor(-41.5310, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.9885, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 347
0.30518632026071346
0
tensor(-84.9399, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-42.1583, device='cuda:0', grad_fn=<NegBackw

0.31034453533022743
0
tensor(-87.0174, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-43.3313, device='cuda:0', grad_fn=<NegBackward0>), tensor(-42.5606, device='cuda:0', grad_fn=<NegBackward0>), tensor(-2.0816, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-87.2598, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-43.3948, device='cuda:0', grad_fn=<NegBackward0>), tensor(-42.7075, device='cuda:0', grad_fn=<NegBackward0>), tensor(-2.1820, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 375
2.9478976283791782
2
This data is wrong to this model, wrong distribution,pass
epoch 376
2.154378728406661
2
This data is wrong to this model, wrong distribution,pass
epoch 377
2.0331313564931945
2
This data is wrong to this model, wrong distribution,pass
epoch 378
2.137938499885056
2
This data is wrong to this model, wrong distribution,pass
epoch 379
1.150625826127612
1
This data is wrong to this model, wrong distribution,pass
epoch 380
0.2907862509164881
0
tensor(-87.3277, device='cuda:0', g

epoch 414
1.345112741168394
1
This data is wrong to this model, wrong distribution,pass
epoch 415
1.2288237842361858
1
This data is wrong to this model, wrong distribution,pass
epoch 416
1.3853973379101543
1
This data is wrong to this model, wrong distribution,pass
epoch 417
1.30074664059497
1
This data is wrong to this model, wrong distribution,pass
epoch 418
2.521543818814823
2
This data is wrong to this model, wrong distribution,pass
epoch 419
0.3048212614051911
0
tensor(-88.2869, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-43.9632, device='cuda:0', grad_fn=<NegBackward0>), tensor(-43.2359, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.9680, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-88.3773, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-44.0341, device='cuda:0', grad_fn=<NegBackward0>), tensor(-43.2352, device='cuda:0', grad_fn=<NegBackward0>), tensor(-2.0284, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 420
1.2123890736060419
1
This data is wrong to this 

epoch 451
2.4032761675479466
2
This data is wrong to this model, wrong distribution,pass
epoch 452
1.1365914317586001
1
This data is wrong to this model, wrong distribution,pass
epoch 453
1.2622563619470153
1
This data is wrong to this model, wrong distribution,pass
epoch 454
1.2977060775785103
1
This data is wrong to this model, wrong distribution,pass
epoch 455
1.445281830054688
1
This data is wrong to this model, wrong distribution,pass
epoch 456
0.27527670553718425
0
tensor(-89.5386, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-44.6042, device='cuda:0', grad_fn=<NegBackward0>), tensor(-44.0145, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.5089, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-89.2935, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-44.4434, device='cuda:0', grad_fn=<NegBackward0>), tensor(-43.9127, device='cuda:0', grad_fn=<NegBackward0>), tensor(-1.5535, device='cuda:0', grad_fn=<AddBackward0>)]
epoch 457
1.0462325072051224
1
This data is wrong to t

epoch 483
1.1487523321813136
1
This data is wrong to this model, wrong distribution,pass
epoch 484
2.2332120699096314
2
This data is wrong to this model, wrong distribution,pass
epoch 485
2.690392808003086
2
This data is wrong to this model, wrong distribution,pass
epoch 486
1.2575901114984789
1
This data is wrong to this model, wrong distribution,pass
epoch 487
1.4494416436916828
1
This data is wrong to this model, wrong distribution,pass
epoch 488
1.1395569643886507
1
This data is wrong to this model, wrong distribution,pass
epoch 489
1.7438694960473033
2
This data is wrong to this model, wrong distribution,pass
epoch 490
3.417656018234299
2
This data is wrong to this model, wrong distribution,pass
epoch 491
2.051812655642038
2
This data is wrong to this model, wrong distribution,pass
epoch 492
0.30188467910869776
0
tensor(-90.4392, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-45.0069, device='cuda:0', grad_fn=<NegBackward0>), tensor(-44.4003, device='cuda:0', grad_fn=<NegBackwa

In [10]:
TT,TF,FT,FF

(163, 2, 0, 315)

In [11]:
torch.save(netD, 'MNIST_NetD_changing2-mode11-point7-repeat.pth')
torch.save(netV, 'MNIST_NetV_changing2-mode11-point7-repeat.pth')
torch.save(netG, 'MNIST_NetG_changing2-mode11-point7-repeat.pth')

In [8]:
# due to storage limits, need to reload and get acc
netD = torch.load('MNIST_NetD_changing2-mode11-point7-repeat.pth') 
netV = torch.load('MNIST_NetV_changing2-mode11-point7-repeat.pth') 
netG = torch.load('MNIST_NetG_changing2-mode11-point7-repeat.pth')

In [9]:
#acc at first for NetD, there exist some issues for netG here, resample will get it right.
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.29s/it]


----------------------------
Train_z, Test_z
PCA: 0.9595
SVD: 0.9577
NetD, Acc at first 0.9684


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.15s/it]


----------------------------
Train_z, Test_z
PCA: 0.9562
SVD: 0.9535
NetG, Acc at first 0.9737
